# Homework 3: Due 11/20
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories. 

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work. 

In [393]:
# Install a pip package in the current Jupyter kernel
import sys

import keras.utils
#!{sys.executable} -m pip install imbalanced-learn delayed
import numpy as np
import scipy
import pandas as pd
import sklearn
import cvxpy
import imblearn
np.random.seed(1)
keras.utils.set_random_seed(1)

# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`. 

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [394]:
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('./garments_worker_productivity.csv')
df=df.drop(columns="date") #Drop the column `date`.

#For each of the categorical attributes, **print out** all the unique elements.
print("Unique values")
for i in [df.quarter,df.department,df.day]:
    print(np.unique(i),i.name)

# For each of the categorical attributes,
# remap the duplicated items, if you find there are typos or spaces among the duplicated items.
df=df.replace(to_replace=["finishing ","sweing"],value=["finishing","sewing"])

#Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable.**
satisfied=[]
for i in df.actual_productivity>=df.targeted_productivity:
    if i:
        satisfied.append(1)
    else:
        satisfied.append(0)
df["Satisfied"]=satisfied

#Drop the columns `actual_productivity` and `targeted_productivity`.
df=df.drop(columns=["actual_productivity","targeted_productivity"])

# Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
print("\nColumn name with na")
for i in df.columns:
     if df[i].isnull().values.any() or df[i].values.any()==None:
         print(i)
# Fill the empty values with 0.
df=df.fillna(0)
df

Unique values
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5'] quarter
['finishing' 'finishing ' 'sweing'] department
['Monday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday' 'Wednesday'] day

Column name with na
wip


,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,Satisfied
0,Quarter1,sewing,Thursday,8,26.16,1108.0,7080,98,0.0,0,0,59.0,1
1,Quarter1,finishing,Thursday,1,3.94,0.0,960,0,0.0,0,0,8.0,1
2,Quarter1,sewing,Thursday,11,11.41,968.0,3660,50,0.0,0,0,30.5,1
3,Quarter1,sewing,Thursday,12,11.41,968.0,3660,50,0.0,0,0,30.5,1
4,Quarter1,sewing,Thursday,6,25.90,1170.0,1920,50,0.0,0,0,56.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10,2.90,0.0,960,0,0.0,0,0,8.0,0
1193,Quarter2,finishing,Wednesday,8,3.90,0.0,960,0,0.0,0,0,8.0,0
1194,Quarter2,finishing,Wednesday,7,3.90,0.0,960,0,0.0,0,0,8.0,0
1195,Quarter2,finishing,Wednesday,9,2.90,0.0,1800,0,0.0,0,0,15.0,0


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [395]:
# Remember to continue the task with your processed data from Exercise 1
## Encoding the categorical attributes:


unique_values=np.unique(df.day).tolist()
df=df.replace(to_replace=unique_values,value=range(unique_values.__len__()))
unique_values=np.unique(df.quarter).tolist()
df=df.replace(to_replace=unique_values,value=range(unique_values.__len__()))
unique_values=np.unique(df.department).tolist()
df=df.replace(to_replace=unique_values,value=range(unique_values.__len__()))
from sklearn.preprocessing import minmax_scale ####THIS IS FOR QUESTION 2.3##############################
np.random.seed(1)
df_cat_columns=["day", "quarter", "department","team"]
numerical_col=set(df.columns)-set(["day", "quarter", "department","team","Satisfied"]) #selected numerical columns
scaled=pd.DataFrame(minmax_scale(df[numerical_col])) #At the start I just scaled all columns that werent categorical or target, rather than scaling them later.
numerical_col=list(range(0,scaled.shape[1]))
##Split the data.
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(pd.concat([scaled,df[df_cat_columns]],axis=1),df["Satisfied"],test_size=0.2, random_state = 15)


C:\Users\gerri\AppData\Local\Temp\ipykernel_40988\3537281072.py:15: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  scaled=pd.DataFrame(minmax_scale(df[numerical_col])) #At the start I just scaled all columns that werent categorical or target, rather than scaling them later.


In [396]:
display(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(957, 12)

(240, 12)

(957,)

(240,)

### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [397]:
# Remember to do this task with your processed data from Exercise 2.1
#https://scikit-learn.org/stable/modules/naive_bayes.html
#Each feature has its own categorical distribution.
#For each feature in the training set, in the feature set, estimate a categorical distribution for each, conditioned on y. For "department", what is the categorical distribution? How many are seiwng and how many are finishing, ect.
import sklearn.naive_bayes as NaB
from sklearn.naive_bayes import CategoricalNB
np.random.seed(1)
CNB_obj=CategoricalNB()                     #initialized object
CNB_obj.fit(X=X_train[df_cat_columns],y=Y_train)
print("Accuracy of mean data {}".format(CNB_obj.score(X_test[df_cat_columns],Y_test))) #Mean accuracy of ``self.predict(X)`` wrt. `y`.
print(sklearn.metrics.classification_report(CNB_obj.predict(X_test[df_cat_columns]),Y_test)) #classification report

Accuracy of mean data 0.7541666666666667
              precision    recall  f1-score   support

           0       0.22      0.61      0.32        23
           1       0.95      0.77      0.85       217

    accuracy                           0.75       240
   macro avg       0.58      0.69      0.59       240
weighted avg       0.88      0.75      0.80       240



In [398]:
numerical_col

[0, 1, 2, 3, 4, 5, 6, 7]

### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [399]:
# Remember to do this task with your processed data from Exercise 2.1
# So, guassian doesn't actually use probability exactly. It uses estimates of standard deviaiton and mean for numerical features, which are proxies of probability, to predict how likely somehting will happen, instead of using the posterior. Personally, I think we should have filled NaN with mean instead, cause now distributions will be shifted.
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
np.random.seed(1)
"All numerical data was already scaled. Look in 2.1"#fits and transforms the inputed data into a zscore.
CNB_obj=GaussianNB()  #initialized object
CNB_obj.fit(X=X_train[numerical_col],y=Y_train)
print("Accuracy of mean data {}".format(CNB_obj.score(X_test[numerical_col],Y_test))) #Mean accuracy of ``self.predict(X)`` wrt. `y`.
print(sklearn.metrics.classification_report(CNB_obj.predict(X_test[numerical_col]),Y_test)) #classification report

Accuracy of mean data 0.7416666666666667
              precision    recall  f1-score   support

           0       0.05      0.75      0.09         4
           1       0.99      0.74      0.85       236

    accuracy                           0.74       240
   macro avg       0.52      0.75      0.47       240
weighted avg       0.98      0.74      0.84       240



In [400]:
X_test

,0,1,2,3,4,5,6,7,day,quarter,department,team
126,0.0,0.055556,0.020132,0.068966,0.000000,0.0,0.0,0.000000,3,1,0,1
305,0.0,0.141204,0.450252,0.655172,0.059727,0.0,0.0,0.013889,2,2,1,9
419,0.0,0.062500,0.020132,0.080460,0.000000,0.0,0.0,0.000000,2,3,0,3
598,0.0,0.268519,0.526520,0.643678,0.017948,0.0,0.0,0.000000,4,0,1,5
161,0.0,0.211806,0.164731,0.327586,0.038362,0.0,0.0,0.013889,1,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...
695,0.0,0.000000,0.379791,0.632184,0.066863,0.0,0.0,0.024444,4,1,1,1
821,0.5,0.259259,0.530972,0.620690,0.036243,0.0,0.0,0.000000,4,2,1,2
211,0.0,0.368056,0.186218,0.586207,0.043422,0.0,0.0,0.012222,0,1,1,11
684,0.0,0.000000,0.138405,0.597701,0.004455,0.0,0.0,0.012500,0,1,1,11


## Exercies 3 - SVM Classifier (40 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [401]:
#selected categorical columns
df_cat_columns=["quarter","department","day","team"]

###ENCODE
hot_mapper=sklearn.preprocessing.OneHotEncoder()
hot_mapper.fit(df[df_cat_columns])
encoded_xtrain=hot_mapper.transform(X_train[df_cat_columns]).toarray() #- For each of the **categorical attribtues**, encode them with **one-hot encoding**.
encoded_xtest=hot_mapper.transform(X_test[df_cat_columns]).toarray() #- For each of the **categorical attribtues**, encode them with **one-hot encoding**.

###Recreate the training data....
X_train=pd.DataFrame(np.concatenate([X_train.drop(columns=["quarter","department","day","team"]),encoded_xtrain],axis=1))
X_test=pd.DataFrame(np.concatenate([X_test.drop(columns=["quarter","department","day","team"]),encoded_xtest],axis=1))

In [402]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.0,0.157407,0.307588,0.367816,0.035205,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.027778,0.000000,0.022989,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.402778,0.507549,0.643678,0.041951,0.000000,0.000000,0.008333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.046296,0.019357,0.091954,0.000000,0.000000,0.000000,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.268519,0.526520,0.643678,0.040394,0.333333,0.011667,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,0.0,0.069444,0.024197,0.091954,0.000000,0.000000,0.000000,0.000000,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
953,0.0,0.125000,0.138405,0.287356,0.027204,0.000000,0.000000,0.012500,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
954,0.0,0.055556,0.024197,0.068966,0.000000,0.000000,0.000000,0.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
955,0.0,0.578704,0.020132,0.264368,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Exercise 3.2 - SVM with Different Kernels (20 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please 
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [403]:
# Remember to do this task with your processed data from Exercise 3.1
from sklearn.svm import SVC
#####Building a dataframe using all attributes... replacing categorical
for i in ["linear","rbf"]:
    clf_li = SVC(kernel=i)          #Build a linear and rbf kernel svm
    clf_li.fit(X=X_train,y=Y_train)
    print(i,"\n",sklearn.metrics.classification_report(y_pred=clf_li.predict(X=X_test),y_true=Y_test))#testing results of both models.

linear 
               precision    recall  f1-score   support

           0       1.00      0.05      0.09        64
           1       0.74      1.00      0.85       176

    accuracy                           0.75       240
   macro avg       0.87      0.52      0.47       240
weighted avg       0.81      0.75      0.65       240

rbf 
               precision    recall  f1-score   support

           0       0.71      0.23      0.35        64
           1       0.78      0.97      0.86       176

    accuracy                           0.77       240
   macro avg       0.75      0.60      0.61       240
weighted avg       0.76      0.77      0.73       240



### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class. 
 - Oversample the **training data**. 
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br > 
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [404]:
# Remember to do this task with your processed data from Exercise 3.1
TRAIN=pd.concat([X_train,Y_train.reset_index(drop=True,inplace=False)],axis=1)
print("Pre_oversample\n",TRAIN.value_counts(subset=Y_train.name)) #Print out frequency of each class
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=21)
X_os_train, Y_os_train=ros.fit_resample(X=TRAIN.drop(columns="Satisfied"),y=TRAIN["Satisfied"])      #Oversampled the data
print("oversampled\n",Y_os_train.value_counts()) #class frequency

Pre_oversample
 Satisfied
1    699
0    258
dtype: int64
oversampled
 0    699
1    699
Name: Satisfied, dtype: int64


## Dont need to rescale the data, since categorical data is one hot, and the data was already scaled before oversampling. Therefore, ROS is going to duplicate SCALED DATA.

In [405]:
for i in ["linear","rbf"]:
    clf_li = SVC(kernel=i)          #Build a linear and rbf kernel svm
    clf_li.fit(X=X_train,y=Y_train)
    print(i,"\n",sklearn.metrics.classification_report(y_pred=clf_li.predict(X=X_test),y_true=Y_test))

linear 
               precision    recall  f1-score   support

           0       1.00      0.05      0.09        64
           1       0.74      1.00      0.85       176

    accuracy                           0.75       240
   macro avg       0.87      0.52      0.47       240
weighted avg       0.81      0.75      0.65       240

rbf 
               precision    recall  f1-score   support

           0       0.71      0.23      0.35        64
           1       0.78      0.97      0.86       176

    accuracy                           0.77       240
   macro avg       0.75      0.60      0.61       240
weighted avg       0.76      0.77      0.73       240



## Exercise 4 - Collaborative Statement (5 points) 
### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

In [406]:
# Got help learning the replace method for Pandas during office hours 11/17/2022 from fellow student. Got help from Piazza. Talked to Dechen during 11/23 office hours.